In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostRegressor, GradientBoostingClassifier
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, StratifiedKFold, cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from xgboost import plot_importance
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from catboost import CatBoostClassifier
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA, TruncatedSVD
import matplotlib.patches as mpatches
from sklearn import base
from sklearn.model_selection import KFold
import collections
# Other Libraries
#from imblearn.datasets import fetch_datasets
from sklearn.pipeline import make_pipeline
from imblearn.pipeline import make_pipeline as imbalanced_make_pipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss
from imblearn.metrics import classification_report_imbalanced
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report
from collections import Counter
from sklearn import preprocessing
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import BaggingClassifier
from tqdm import tqdm
from sklearn.cluster import KMeans
#import category_encoders as ce
import warnings
import lightgbm as lgb
from pandas import DataFrame
#from geopy.geocoders import Nominatim
#from geopy.distance import vincenty
#from pygeocoder import Geocoder
warnings.filterwarnings("ignore")

C:\Users\Emma\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [2]:
# Dataset directories
dir_train = 'trainx.csv'
dir_test = 'testx.csv'
dir_sub = 'sub.csv'

In [3]:
df_train = pd.read_csv(dir_train, header=None)
df_train = pd.DataFrame(df_train[0].str.split(',').tolist())


df_test = pd.read_csv(dir_test, header=None)
df_test = pd.DataFrame(df_test[0].str.split(',').tolist())


df_sub = pd.read_csv(dir_sub, header=None)
df_sub = pd.DataFrame(df_sub[0].str.split(',').tolist())

In [4]:
df_train.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,Customer Id,YearOfObservation,Insured_Period,Residential,Building_Painted,Building_Fenced,Garden,Settlement,Building Dimension,Building_Type,Date_of_Occupancy,NumberOfWindows,Geo_Code,Claim
1,H14663,2013,1,0,N,V,V,U,290,1,1960,.,1053,0
2,H2037,2015,1,0,V,N,O,R,490,1,1850,4,1053,0
3,H3802,2014,1,0,N,V,V,U,595,1,1960,.,1053,0
4,H3834,2013,1,0,V,V,V,U,2840,1,1960,.,1053,0


In [5]:
df_train = df_train.rename(columns=df_train.iloc[0])
df_test = df_test.rename(columns=df_test.iloc[0])
df_sub = df_sub.rename(columns=df_sub.iloc[0])

In [6]:
df_train = df_train.drop(df_train.index[0])
df_test = df_test.drop(df_test.index[0])
df_sub = df_sub.drop(df_sub.index[0])

In [7]:
df_train.head(10)

,Customer Id,YearOfObservation,Insured_Period,Residential,Building_Painted,Building_Fenced,Garden,Settlement,Building Dimension,Building_Type,Date_of_Occupancy,NumberOfWindows,Geo_Code,Claim
1,H14663,2013,1,0,N,V,V,U,290,1,1960,.,1053,0
2,H2037,2015,1,0,V,N,O,R,490,1,1850,4,1053,0
3,H3802,2014,1,0,N,V,V,U,595,1,1960,.,1053,0
4,H3834,2013,1,0,V,V,V,U,2840,1,1960,.,1053,0
5,H5053,2014,1,0,V,N,O,R,680,1,1800,3,1053,0
6,H4977,2012,1,0,V,N,O,R,535,1,1980,3,1143,0
7,H7390,2012,1,0,N,V,V,U,2830,1,1988,.,1143,0
8,H14488,2015,1,0,N,V,V,U,4952,1,1988,.,1160,0
9,H19355,2014,1,0,V,N,O,R,2735,1,2013,3,1173,1
10,H18601,2015,1,0,V,N,O,R,520,1,2011,2,1224,0


In [8]:
df_train['YearOfObservation'] = df_train['YearOfObservation'].astype(int)
df_train['Insured_Period'] = df_train['Insured_Period'].astype(float)
df_train['Residential'] = df_train['Residential'].astype(int)
df_train['Building_Type'] = df_train['Building_Type'].astype(int)
df_train['Claim'] = df_train['Claim'].astype(int)

df_train['Date_of_Occupancy'] = pd.to_numeric(df_train['Date_of_Occupancy'],errors='coerce')
df_train['NumberOfWindows'] = pd.to_numeric(df_train['NumberOfWindows'],errors='coerce')
df_train['Geo_Code'] = pd.to_numeric(df_train['Geo_Code'],errors='coerce')
df_train['Building Dimension'] = pd.to_numeric(df_train['Building Dimension'],errors='coerce')

In [9]:
df_test['YearOfObservation'] = df_test['YearOfObservation'].astype(int)
df_test['Insured_Period'] = df_test['Insured_Period'].astype(float)
df_test['Residential'] = df_test['Residential'].astype(int)
df_test['Building_Type'] = df_test['Building_Type'].astype(int)

df_test['Date_of_Occupancy'] = pd.to_numeric(df_test['Date_of_Occupancy'],errors='coerce')
df_test['NumberOfWindows'] = pd.to_numeric(df_test['NumberOfWindows'],errors='coerce')
df_test['Geo_Code'] = pd.to_numeric(df_test['Geo_Code'],errors='coerce')
df_test['Building Dimension'] = pd.to_numeric(df_test['Building Dimension'],errors='coerce')

In [10]:
df_train.dtypes

Customer Id            object
YearOfObservation       int32
Insured_Period        float64
Residential             int32
Building_Painted       object
Building_Fenced        object
Garden                 object
Settlement             object
Building Dimension    float64
Building_Type           int32
Date_of_Occupancy     float64
NumberOfWindows       float64
Geo_Code              float64
Claim                   int32
dtype: object

In [11]:
df_test.dtypes

Customer Id            object
YearOfObservation       int32
Insured_Period        float64
Residential             int32
Building_Painted       object
Building_Fenced        object
Garden                 object
Settlement             object
Building Dimension    float64
Building_Type           int32
Date_of_Occupancy     float64
NumberOfWindows       float64
Geo_Code              float64
dtype: object

In [13]:
df_train.head(10)

,Customer Id,YearOfObservation,Insured_Period,Residential,Building_Painted,Building_Fenced,Garden,Settlement,Building Dimension,Building_Type,Date_of_Occupancy,NumberOfWindows,Geo_Code,Claim
1,H14663,2013,1.0,0,N,V,V,U,290.0,1,1960.0,NaN,1053.0,0
2,H2037,2015,1.0,0,V,N,O,R,490.0,1,1850.0,4.0,1053.0,0
3,H3802,2014,1.0,0,N,V,V,U,595.0,1,1960.0,NaN,1053.0,0
4,H3834,2013,1.0,0,V,V,V,U,2840.0,1,1960.0,NaN,1053.0,0
5,H5053,2014,1.0,0,V,N,O,R,680.0,1,1800.0,3.0,1053.0,0
6,H4977,2012,1.0,0,V,N,O,R,535.0,1,1980.0,3.0,1143.0,0
7,H7390,2012,1.0,0,N,V,V,U,2830.0,1,1988.0,NaN,1143.0,0
8,H14488,2015,1.0,0,N,V,V,U,4952.0,1,1988.0,NaN,1160.0,0
9,H19355,2014,1.0,0,V,N,O,R,2735.0,1,2013.0,3.0,1173.0,1
10,H18601,2015,1.0,0,V,N,O,R,520.0,1,2011.0,2.0,1224.0,0


In [15]:
df_train.to_csv('tr.csv', index=False)
df_test.to_csv('tes.csv', index=False)